In [1]:
# Packages

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime


### Data pre-processing :

In [3]:
# Functions for preprocessing the raw data, the steps are :
# Event aggregation
# Sorting by time
def data_preprocessing(data1,data2,data3,data4):
          # Loading files
          device_df=pd.read_csv(data1)
          logon_df=pd.read_csv(data2)
          email_df=pd.read_csv(data3)
          file_df=pd.read_csv(data4)
          


          # Show shape
          print("Shape of ",data1," : ",device_df.shape,"\n") 
          print("Shape of ",data2," : ",logon_df.shape,"\n") 
          print("Shape of ",data3," : ",email_df.shape,"\n") 
          print("Shape of ",data4," : ",file_df.shape,"\n") 
          
          # Event aggregation
          merged_df=pd.concat([device_df,logon_df,email_df,file_df])


          # Replacing the date column with different format
          #merged_df['date']=pd.to_datetime(merged_df['date'],dayfirst=False)

          # Sorting by date
          #sorted_df=merged_df.sort_values(by=['date'])
          #print(" First rows of merged Dataframes sorted by date : ","\n",sorted_df.head(),"\n")
          print(" First rows of merged Dataframes sorted by date : ","\n",merged_df.head(),"\n")
          
          return merged_df

In [4]:
merged_df=data_preprocessing('device.csv','logon.csv','email.csv','file.csv')

### Feature Engineering :

In [ ]:
def prepare_data_for_a_user(user,user_abnormal_events,sorted_df):
          
          # For User = user
          user_1=sorted_df[sorted_df['user']==user]
          # Show
          print("First rows of ",user," events : ","\n",user_1.head(),"\n")
          print('Shape : ',user_1.shape,"\n")
          
          
          
          # Importing the abnormal events 
          events_user_1_abnormal=pd.read_csv(user_abnormal_events,header=None)
          
          
          # We keep only normal events 
          # The id of abnormal events in a list 
          id_abnormal=events_user_1_abnormal[1]
          # We drop the corresponding abnormal events
          user_1_normal=user_1 
          for id in id_abnormal:
                    user_1_normal=user_1_normal[~(user_1_normal['id']==id)]
          # Show
          print("First rows of ",user," normal events : ","\n",user_1_normal.head(),"\n")
          print('shape : ',user_1_normal.shape)



          # We take only the abnormal events
          user_1_abnormal=pd.DataFrame([],columns=['id','date','user','pc','activity'])
          for id in id_abnormal:
                    user_1_abnormal=pd.concat([user_1_abnormal,user_1[(user_1['id']==id)]])   
          user_1_abnormal
          print("First rows of ",user," abnormal events : ","\n",user_1_abnormal.head(),"\n")
          print('Shape : ',user_1_abnormal.shape)
          
          
          # Creating the DataFrame of vector feature of all events
          table_feature_vector_user_1=raw_data_to_Table_of_Feature_Vectors(user_1)
          print("First rows of ",user," Table of Feature Vectors with Days including normal and abnormal events : ","\n",table_feature_vector_user_1,"\n")
          print("Shape : ",table_feature_vector_user_1.shape)
          

          # Creating the DataFrame of vector feature for the abnormal events
          table_feature_vector_user_1_abnormal=raw_data_to_Table_of_Feature_Vectors(user_1_abnormal)
          # We will remove the days with abnormal events and keep only days with normal events in table_feature_vector_user_1_normal
          # and we will keep all events (normal and abnormal) in table_feature_vector_user_1
          abnormal_day=table_feature_vector_user_1_abnormal['date']
          print("Days that have abnormal events : ",'\n',abnormal_day,"\n")

          # We will create a table with only days with normal events
          table_feature_vector_user_1_normal=table_feature_vector_user_1
          for day in abnormal_day:
                    table_feature_vector_user_1_normal=table_feature_vector_user_1_normal[~(table_feature_vector_user_1_normal['date']==day)]
          print("First rows of ",user," Table of Feature Vectors with Days including only normal events : ","\n",table_feature_vector_user_1_normal,"\n")
          print("Shape : ",table_feature_vector_user_1_normal.shape)
          

          # Creating a vector insiderthreat(like y in tutoriels) with (1/0) where the day is abnormal/normal  if in this day have abnormal events 
          date_for_insiderthreat=table_feature_vector_user_1['date']
          date_for_insiderthreat.tolist()
          insiderthreat = date_for_insiderthreat.isin(abnormal_day)*1
          insiderthreat=insiderthreat.tolist()
          print("insiderthreat vector is the true output(y), the days where there was abnormal events : ","\n",insiderthreat)
          
          
          
          
          # We drop the date columns for model training purpuse
          table_feature_vector_user_1_normal=table_feature_vector_user_1_normal.drop(['date'],axis=1)
          table_feature_vector_user_1=table_feature_vector_user_1.drop(['date'],axis=1)


          return table_feature_vector_user_1, table_feature_vector_user_1_normal, insiderthreat

In [2]:
email_df=pd.read_csv('email.csv')